In [ ]:
import tempfile
import os
import base64

DOC_URL = ""
PROJECT_ID = "aidt-genie"
REGION = "us-central1"
os.environ["PROJECT_ID"] = PROJECT_ID
os.environ["OPENAI_API_KEY"] = "asdf"

import sys
sys.path.append("../../common/src")
sys.path.append("../src")

from google.cloud import storage

from services.query.query_service import datasource_from_url
from services.query.data_source import DataSource

storage_client = storage.Client(project=PROJECT_ID)

# get datasource class for DOC_URL
data_source = datasource_from_url(DOC_URL, None, storage_client)


In [ ]:

docs_processed = []
with tempfile.TemporaryDirectory() as temp_dir:
  data_source_files = data_source.download_documents(DOC_URL, temp_dir)

  # counter for unique index ids
  index_base = 0

  for data_source_file in data_source_files:
    doc_name = data_source_file.doc_name
    index_doc_url = data_source_file.src_url
    doc_filepath = data_source_file.local_path

    print(f"processing [{doc_name}]")
    print(index_doc_url)
    print(data_source_file.gcs_path)
    
    slide_chunks = data_source.chunk_document_multi(doc_name, data_source_file.gcs_path, doc_filepath)
    for slide in slide_chunks:
      print(slide["image_url"])
      # image_bytes = base64.b64decode(slide["image_b64"])
      # with open("image.png", "wb") as f:
      #     f.write(image_bytes)
    
    # # cleanup temp local file
    # os.remove(doc_filepath)